###### Для части 1.

In [1]:
import sqlite3
import re
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

### 1. Как посмотреть вхождения в корпусе

In [2]:
db = sqlite3.connect('ALLP.db')
cur = db.cursor()
cur.execute('''SELECT author.band, song.songname, song.year, token.id, token.song_id, token.line, token.token, token.lem, token.pos, token.in_sent FROM token
            JOIN song ON song.id = token.song_id
        JOIN author ON author.id = song.author_id''')
tokens = cur.fetchall()
db.close()

In [3]:
tokens[10]

('Loboda', 'Твои глаза', 2016, 1557, 18, 1, 'на', 'на', 'PR', 3)

In [4]:
def parsebase(tokens):
    tdictlist = []
    for token in tokens:
        tok = {}
        tok['artist']=token[0]
        tok['songname']=token[1]
        tok['year']=token[2]
        tok['id']=token[3]
        tok['song_id']=token[4]
        tok['line']=token[5]
        tok['token']=token[6]
        tok['lem']=token[7]
        tok['pos']=token[8]
        tok['in_sent']=token[9]
        tdictlist.append(tok)
        
    return tdictlist  

In [5]:
basetokens = parsebase(tokens)

In [6]:
basetokens[1100]

{'artist': 'Loboda',
 'songname': 'Я влюбилась',
 'year': 2008,
 'id': 2647,
 'song_id': 28,
 'line': 9,
 'token': 'постель',
 'lem': 'постель',
 'pos': 'S',
 'in_sent': 5}

#### Функции для обработки запроса

Предобработка для вхождений типа 'дождям', чтобы получить начальную форму. Мой алгоритм обрабатывает три типа запросов:<br>
<ol><li>С двойными кавычками в начале - ищет полное совпадение с вводом (например, "дождям")</li><li>Без кавычек, начальная форма - ищет все вхождения токена в любых формах</li><li>По частеречному тегу (использованы теги <a href='https://yandex.ru/dev/mystem/doc/grammemes-values.html/'>mystem</a>)</li></ol>
<i>Precheck</i> при помощи <i>checkform</i> заменяет неначальные формы не в кавычках в начальные формы для обработки по типу 2.<br><br>*Также обрабатываются запросы типа 'сосну+S', т.е. с неначальной формой (но можно и с начальной) и с частеречным тегом. Главное - <b>без</b> кавычек

In [7]:
def precheck(query):
    query.strip(' ')
    queries = query.split(' ')
    checked = []
    for item in queries:
        check = checkform(item)
        checked.append(check)
    return checked

In [8]:
def checkform(form):
    if not form.startswith(('A', 'C', 'I', 'N', 'P', 'S', 'V', '"')):
        lex = m.analyze(form)[0]['analysis'][0]['lex']
        if len(form.split('+'))==1:
            return lex
        else:
            return lex+'+'+form.split('+')[1]
            
    else:
        return form

В <i>search</i> подается список запросов, запросы проверяются в базе при помощи <i>checktype</i>. Первые вхождения проверяются отдельно от всех последующих, потому что без наличия первого невозможны другие.<br>NB в поиске я использую шаг от начальной формы не больше 2 (по заданию), но в целом шаг (step) можно делать сколько угодно удаленным.

In [9]:
def search(queries, tdictlist):  # по очереди итерируюсь, сколько нужно, убирая кандидатов на каждом шаге
    if len(queries)==1:
        last = checktype(queries[0], tdictlist, num=0) 
        if last:
            return last
    elif len(queries)==2:
        first = checktype(queries[0], tdictlist, num=0) 
        if first:
            last = checktype(queries[1], tdictlist, num=1, found=first)
            if last:
                return last
    else:
        first = checktype(queries[0], tdictlist, num=0)
        if first:
            second = checktype(queries[1], tdictlist, num=1, found=first)
            if second:
                last = checktype(queries[2], tdictlist, num=2, found=second)
                if last:
                    return last

In [10]:
def checktype(query, tdictlist, num=0, found=[]):
    
    foundtokens = []
    
    if num==0:
        if query.startswith('"'):
            for num, item in enumerate(tdictlist):
                if item['token']==query.strip('"'):
                    s_id = item['song_id']
                    token_id = item['id']
                    line = item['line']
                    foundtokens.append([num, s_id, line])  
        elif not query.startswith(('A', 'C', 'I', 'N', 'P', 'S', 'V')):
            if len(query.split('+'))==1:
                for num, item in enumerate(tdictlist):
                    if item['lem']==query:
                        s_id = item['song_id']
                        token_id = item['id']
                        line = item['line']
                        foundtokens.append([num, s_id, line])
            else:
                que, q_pos = query.split('+')
                for num, item in enumerate(tdictlist):
                    if item['lem']==que and item['pos'] == q_pos:
                        s_id = item['song_id']
                        token_id = item['id']
                        line = item['line']
                        foundtokens.append([num, s_id, line])
                
        else:
            for num, item in enumerate(tdictlist):
                if item['pos']==query:
                    s_id = item['song_id']
                    token_id = item['id']
                    line = item['line']
                    foundtokens.append([num, s_id, line])
        return foundtokens
    
    else:
        step = num
        nextf = []
        for item in found:
            num_inlist = item[0]
            s_id = item[1]
            if num_inlist+step<len(tdictlist):  # проверка, есть ли в списке с данными следующий элемент и что проверять
                if query.startswith('"'):
                    if tdictlist[num_inlist+step]['token']==query.strip('"'):
                            s2id = tdictlist[num_inlist+step]['song_id']
                            line = tdictlist[num_inlist+step]['line']
                            if s_id ==s2id:
                                item.append(line)
                                nextf.append(item)            
                elif not query.startswith(('A', 'C', 'I', 'N', 'P', 'S', 'V')):
                    if len(query.split('+'))==1:
                        if tdictlist[num_inlist+step]['lem']==query:
                            s2id = tdictlist[num_inlist+step]['song_id']
                            line = tdictlist[num_inlist+step]['line']
                            if s_id ==s2id:
                                item.append(line)
                                nextf.append(item)
                    else:
                        que, q_pos = query.split('+')
                        if tdictlist[num_inlist+step]['lem']==que and tdictlist[num_inlist+step]['pos'] == q_pos:
                            s2id = tdictlist[num_inlist+step]['song_id']
                            line = tdictlist[num_inlist+step]['line']
                            if s_id ==s2id:
                                item.append(line)
                                nextf.append(item)                        
                else:
                    if tdictlist[num_inlist+step]['pos']==query:
                        s2id = tdictlist[num_inlist+step]['song_id']
                        line = tdictlist[num_inlist+step]['line']
                        if s_id ==s2id:
                            item.append(line)
                            nextf.append(item)
        return nextf      

При помощи <i>index2df</i> индексы вхождений, полученные на предыдущем этапе, превращаются в строки с вхождениями

In [29]:
def index2df(indexlist, tdictlist):
    fordf = []
    for item in indexlist:
        numfromlist = item[0]
        s_id = item[1]
        line = item[2]
        string = []
        for token in tdictlist[numfromlist-25:]:
            if token['song_id']==s_id:
                if token['line']==line or token['line']==line+1:
                    string.append(token['token'])
        songitems = []
        songitems.append(tdictlist[numfromlist]['artist'])
        songitems.append(tdictlist[numfromlist]['songname'])
        songitems.append(tdictlist[numfromlist]['year'])
        songitems.append(' '.join([item for item in string]))
        fordf.append(songitems)
    return fordf

Финальный датафрэйм

In [12]:
def getdf(query, tdictlist):
    checked = precheck(query)
    lists = search(checked, tdictlist)
    if lists:
        fordf = index2df(lists, tdictlist)
        df = pd.DataFrame(fordf, columns=['artist', 'songname', 'year', 'lyrics'])
        return df

#### Примеры

In [52]:
getdf('"Нас" PART догонят+V', basetokens)  # сочетание всех типов

,artist,songname,year,lyrics
0,t.A.T.u.,Нас Не Догонят,2001,Нас не догонят Нас не догонят нас не догонят
1,t.A.T.u.,Нас Не Догонят,2001,Нас не догонят нас не догонят Только скажи дал...


In [25]:
getdf('CONJ SPRO "влюблен"', basetokens)  

,artist,songname,year,lyrics
0,Иван Дорн,Северное сияние,2012,День был холодный а я влюблен Нас было трое жа...


In [30]:
getdf('A S V', basetokens)

,artist,songname,year,lyrics
0,Loboda,Твои глаза,2016,Неловкое молчание повисло Слишком слишком близ...
1,Loboda,Случайная,2017,Простые ответы увидимся снова Заклеив конверты...
2,Loboda,40 градусов,2012,Его неземные слова допила до дна Держи меня но...
3,Loboda,"Потому, что люблю",2005,Слепая тоска несет Измученный крик надежды
4,Loboda,Чёрный ангел,2005,Чёрным ангелом в ночи тихим ветром постучи Ты ...
...,...,...,...,...
1055,Потап и Настя,У нас на районе,2009,А по району ходят бродят классные цыпочки Корч...
1056,Потап и Настя,У нас на районе,2009,Дворовыми басами завывают с нами
1057,Потап и Настя,Позвони своей зае,2009,Комната пустая шторы закрываю Больше не вернёш...
1058,Потап и Настя,"Чипсы, чиксы, лавандос",2009,Виртуальные кенты говорят что я лох Потому что...


In [39]:
getdf('клубом дымом', basetokens)

,artist,songname,year,lyrics
0,Децл,Меломанов плейлисты,2018,Клубы дыма стробоскопы лазеры прожектора Здесь...
1,Bad Balance,Городская тоска,1996,Под густыми клубами дыма улицы темнеют Город п...


In [40]:
getdf('"Я" "любила"', basetokens)

,artist,songname,year,lyrics
0,Винтаж,Ева,2009,Я любила тебя Беги от меня


In [41]:
getdf('"Я" "любил"', basetokens)

,artist,songname,year,lyrics
0,Тима Белорусских,Привычка убегать,2019,Я любил бы их всех их всех
1,Скриптонит,Зеркала,2017,Я любил в этом страсть а не пошлость Курица ра...


In [42]:
getdf('я "любил"', basetokens)

,artist,songname,year,lyrics
0,Тима Белорусских,Привычка убегать,2019,Я любил бы их всех их всех
1,Скриптонит,Зеркала,2017,Я любил в этом страсть а не пошлость Курица ра...
2,Баста,Вселенная,2013,Но я любил так будто клялся будто дал обет Пру...
3,Валерий Меладзе,Потерян И Не Найден,2014,А я любил ее мучительно и нежно каждый день те...
4,Валерий Меладзе,Потерян И Не Найден,2014,Ведь я любил ее мучительно и нежно каждый день...


In [51]:
getdf('я+SPRO люблю', basetokens)

,artist,songname,year,lyrics
0,Loboda,Чёрный ангел,2005,Я люблю тебя я люблю тебя я люблю тебя Отпусти...
1,Loboda,Чёрный ангел,2005,Я люблю тебя я люблю тебя я люблю тебя Отпусти...
2,Loboda,Чёрный ангел,2005,Я люблю тебя я люблю тебя я люблю тебя Отпусти...
3,Loboda,"Там, за облаками",2005,Теперь ты знаешь как я люблю Меня возьми меня ...
4,Егор Крид,Потрачу,2017,Люблю и ненавижу я люблю и ненавижу С тобой я ...
...,...,...,...,...
68,Винтаж,Kill.Dim.Bil.,2007,Просто дура просто я любила
69,Наталья Ветлицкая,Пусть будет,1994,Что ты меня любишь и ждёшь Мне сегодня просто ...
70,ДДТ,В ресторане,2003,Прости меня я люблю твои овалы и тени И вдруг ...
71,Банд’Эрос,"Танцуй, Рублевка",2006,Я люблю эти улицы с грязным лицом Моё место зд...


In [45]:
getdf('спит S', basetokens)

,artist,songname,year,lyrics
0,Егор Крид,Папина дочка,2015,Я обожаю то как она спит ноги руки на мне это ...
1,Shortparis,Страшно,2018,Спят сыновья Молчит семья
2,Shortparis,Поломало,2019,Пока спала Беда текла
3,Shortparis,Отвечай за слова,2019,Спит пространство двора и спит детвора Тихо и ...
4,Shortparis,Отвечай за слова,2019,Спит пространство двора и спит детвора Тихо и ...
5,Shortparis,Отвечай за слова,2019,Спит рабочий и с ним вторая жена Спит как будт...
6,Shortparis,"Раз, два, три",2017,В ванной спит жених В женщине дитя
7,Грибы,Копы,2016,Это принципиально нету времени спать Руки в во...
8,Баста,Раз и навсегда,2006,Я мог не спать ночами находясь в тумане Я кури...
9,Дельфин,713,2018,Спящих людей которых не знаю Они не проснутся ...


In [50]:
getdf('S петь', basetokens)

,artist,songname,year,lyrics
0,Loboda,Твои глаза,2016,
1,Loboda,Дай мне,2005,Суета пело тело мое и душа Но сказала я Стоп н...
2,монеточка,90,2018,Ведь только Ласковый Май пел по радио А я живу...
3,монеточка,Козырный туз,2016,Озверели оборзели в интернете песни пели Ах пр...
4,Тима Белорусских,Песня-SOS,2019,Благодаря тебе моя душа поет Больше не надо ни...
5,FACE,САЛАМ,2018,Заряжен укороченный АК поёт как флейта Все мои...
6,Shortparis,Любовь,2017,Водой Пой дева пой
7,Shortparis,Любовь,2017,Пой дева пой Он с тобой
8,Shortparis,Любовь,2017,Ты здесь прикройся рукой Пой сука пой
9,Shortparis,Любовь,2017,Пой сука пой Что живой


In [53]:
getdf('фейсбук', basetokens)

,artist,songname,year,lyrics
0,монеточка,Пост-пост,2018,Без аккаунта в фейсбуке без диплома и паспорта...
1,монеточка,Козырный туз,2016,За все репосты и подписки в фейсбуке Коли мужи...
2,Ленинград,Вояж,2017,Чтобы фоточки в Фейсбуке залепить как говоритс...
